In [17]:
import yfinance as yf
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [18]:
#Set-up end range

yesterday = datetime.now() - timedelta(1)
Last_Week = datetime.now() - timedelta(3)

exclude_weekend = [6,7]
weekdays  = [5,6]
date_range = []

for days in exclude_weekend:
    
    if days == datetime.isoweekday(yesterday):
    
        Reporting_Date = Last_Week
    
    else:
    
        Reporting_Date = yesterday
    
    Year = datetime.strftime(Reporting_Date, '%Y')
    Month = datetime.strftime(Reporting_Date, '%m')
    Day = datetime.strftime(Reporting_Date, '%d')

date= Year +'-'+ Month + '-' + Day

#set-up start range

master_start_date = Reporting_Date - timedelta(180)

year_start = datetime.strftime(master_start_date, '%Y')
month_start = datetime.strftime(master_start_date, '%m')
day_start = datetime.strftime(master_start_date, '%d')

start_date = year_start +'-'+ month_start +'-'+ day_start

print(date, start_date)

2024-06-24 2023-12-27


In [19]:
tickers = ['DAX',
          'SAP.DE',
          'SIE.DE',
          'ALV.DE',
          'AIR.DE',
          'DTE.DE',
          'MBG.DE',
          'BAS.DE',
          'IFX.DE',
          'DHL.DE',
          'ADS.DE']

In [20]:
prices = yf.download(tickers,  # list of tickers
            period = "1y",         # time period
            interval = "1d",       # trading interval
            prepost = False,       # download pre/post market hours data?
            repair = True)  

[                       0%                       ]DHL.DE: fixed 18/18 value=0 errors in 1d price data
[*********             18%                       ]  2 of 11 completedALV.DE: fixed 3/3 value=0 errors in 1h price data: [Timestamp('2024-06-20 09:00:00+0200', tz='Europe/Berlin'), Timestamp('2024-06-21 09:00:00+0200', tz='Europe/Berlin'), Timestamp('2024-06-24 09:00:00+0200', tz='Europe/Berlin')]
ALV.DE: fixed 6/6 value=0 errors in 1d price data
MBG.DE: fixed 3/3 value=0 errors in 1h price data: [Timestamp('2024-06-20 09:00:00+0200', tz='Europe/Berlin'), Timestamp('2024-06-21 09:00:00+0200', tz='Europe/Berlin'), Timestamp('2024-06-24 09:00:00+0200', tz='Europe/Berlin')]
ADS.DE: fixed 3/3 value=0 errors in 1h price data: [Timestamp('2024-06-20 09:00:00+0200', tz='Europe/Berlin'), Timestamp('2024-06-21 09:00:00+0200', tz='Europe/Berlin'), Timestamp('2024-06-24 09:00:00+0200', tz='Europe/Berlin')]BAS.DE: fixed 3/3 value=0 errors in 1h price data: [Timestamp('2024-06-20 09:00:00+0200', tz=

In [21]:
prices['Adj Close']
prices = prices.dropna()

In [22]:
bars_data = {}

for stock in tickers:
    bars_data[stock] = np.array([prices[('Open', stock)].values, prices[('High', stock)].values, prices[('Low', stock)].values, prices[('Adj Close', stock)].values]).T

In [23]:
bars_data

{'DAX': array([[29.67000008, 29.81500053, 29.39999962, 28.92252731],
        [29.72999954, 30.01199913, 29.70000076, 29.07849693],
        [29.92000008, 30.06999969, 29.73500061, 29.31245041],
        [29.13999939, 29.27499962, 28.93600082, 29.08428001],
        [29.57999992, 29.6609993 , 29.47999954, 29.50275803],
        [29.52000046, 29.52499962, 29.42000008, 29.41806602],
        [29.28000069, 29.28000069, 28.94000053, 28.95475006],
        [28.62999916, 28.62999916, 28.27499962, 28.35692215],
        [28.58499908, 28.84000015, 28.58499908, 28.72558212],
        [28.81999969, 29.01399994, 28.79000092, 28.82522202],
        [29.09000015, 29.25      , 29.07999992, 29.13409805],
        [29.77000046, 30.01099968, 29.77000046, 29.89134598],
        [30.38999939, 30.45999908, 30.35000038, 30.34967804],
        [30.30999947, 30.30999947, 30.30999947, 30.20022202],
        [30.17000008, 30.43000031, 30.12000084, 30.31978798],
        [30.29999924, 30.40999985, 30.29000092, 30.29986   ],
 

In [24]:
index = 'DAX'

holdings = [
    'SAP.DE',
    'SIE.DE',
    'ALV.DE',
    'AIR.DE',
    'DTE.DE',
    'MBG.DE',
    'BAS.DE',
    'IFX.DE',
    'DHL.DE',
    'ADS.DE'
]

In [25]:
corr_data = {}
stock_ret_store = {}

df_data = bars_data[index]
reshape_df = df_data.reshape(-1, 4)
index_ret = np.log(reshape_df[1:, :]/reshape_df[:-1, :])

In [27]:
for stock in holdings:
    df_data = bars_data[stock]
    reshape_df = df_data.reshape(-1, 4)
    stock_ret = np.log(reshape_df[1:, :]/reshape_df[:-1, :])
    stock_ret_store[stock] = stock_ret
    
    corr_data[stock] = np.corrcoef(index_ret, stock_ret)

In [28]:
corr_data

{'SAP.DE': array([[ 1.        , -0.74117603, -0.16501812, ...,  0.47863053,
         -0.00391433, -0.76965759],
        [-0.74117603,  1.        ,  0.74871611, ...,  0.07700344,
         -0.46564414,  0.15077306],
        [-0.16501812,  0.74871611,  1.        , ...,  0.71211023,
         -0.87941722, -0.4977479 ],
        ...,
        [ 0.47863053,  0.07700344,  0.71211023, ...,  1.        ,
         -0.87881019, -0.84792447],
        [-0.00391433, -0.46564414, -0.87941722, ..., -0.87881019,
          1.        ,  0.53216536],
        [-0.76965759,  0.15077306, -0.4977479 , ..., -0.84792447,
          0.53216536,  1.        ]]),
 'SIE.DE': array([[ 1.        , -0.74117603, -0.16501812, ...,  0.78250655,
         -0.80044955,  0.61533567],
        [-0.74117603,  1.        ,  0.74871611, ..., -0.18286553,
          0.21366785, -0.01876999],
        [-0.16501812,  0.74871611,  1.        , ...,  0.48493063,
         -0.4590741 ,  0.64811125],
        ...,
        [ 0.78250655, -0.18286553,

In [29]:
realized_estimate = {}

for stock in holdings:
    df_data = bars_data[stock]
    #Calculate realized volatility
    intraday_vol = np.log(df_data[1: ,1]/df_data[1: ,0]) * \
                   np.log(df_data[1: ,1]/df_data[1: ,3]) + \
                   np.log(df_data[1: ,2]/df_data[1: ,0]) * \
                   np.log(df_data[1: ,2]/df_data[1: ,3])
    intraday_vol = (np.sqrt(intraday_vol) * 100 * 16)**2
    overnight_vol = (np.abs(np.log(df_data[1:,0]/df_data[:-1,-1]))*100*16)**2
    realized_vol = np.sqrt(intraday_vol + overnight_vol)
    
    #Calculate estimate for short medium and long term realized vol
    short_term_vol = 
    

C:\Users\GLO055\AppData\Local\Temp/ipykernel_17688/800056446.py:10: RuntimeWarning: invalid value encountered in sqrt
  intraday_vol = (np.sqrt(intraday_vol) * 100 * 16)**2


In [30]:
intraday_vol

array([ 330.85314127,  877.42656224,  387.35981557, 2434.96822969,
        841.41507619,  278.22014514,  236.45291578,  813.66880519,
        317.55406974,  865.66619509,  621.75902144,  263.30100726,
        148.86065701,  380.48440947,   78.50574303,  253.92033215,
        283.45708657,  156.2085068 ,  446.84736968, 2534.41508836,
        398.49171304,  572.43980934, 1253.76988098,  551.19618586,
                 nan,  564.92691852, 1849.3952809 ,  564.43551841,
        270.9300673 ,  207.25758425,  237.22033603,  617.62381919,
        243.14782848,  416.16178538, 1161.53858516,  477.45642854,
       1225.61898502,  325.88898009, 1672.28224004, 1354.85658277,
       6656.33973056, 1044.35534925, 1514.3707887 ,  450.18487354,
        725.28933337,  626.4145916 ,  663.85614777,  509.11361397,
        734.4729751 ,  543.36601695,  810.61398336,  498.50679128,
        764.350622  ,   67.15431741, 2360.71636232, 1127.21016537,
        341.39386029,  617.44322754,  517.68010804,  500.73245